# AUDITORÍA VALORES CRÍTICOS RADIOLOGÍA

In [ ]:
import pandas as pd
import numpy as nu
import regex as re

In [ ]:
# Cargar archivos
df_valores_criticos = pd.read_excel("Oportunidad-Radiología.xlsx")
df_hc_texto = pd.read_excel("HCText-Radiologia-V1.xlsx")

# Renombrar columnas para facilidad
df_hc_texto = df_hc_texto.rename(columns={
    "EPIINAHIS": "Historia clinica",
    "TO_CHAR(REGCLIFCH,'DD/MM/YYYYHH24:MI:SS')": "Fecha nota HC",
    "CASEWHENREGCLIRTFISNULLTHENREGCLITXTELSEREGCLIRTFEND": "Texto HC"
})

# Normalizar texto y fechas
df_hc_texto["PRONOM"] = df_hc_texto["PRONOM"].str.upper()
df_hc_texto["ESPNOM"] = df_hc_texto["ESPNOM"].str.upper()
df_hc_texto["Fecha nota HC"] = pd.to_datetime(df_hc_texto["Fecha nota HC"], format="%d/%m/%Y %H:%M:%S", errors='coerce')
df_valores_criticos["Fecha nota"] = pd.to_datetime(df_valores_criticos["Fecha nota"], errors='coerce').dt.floor('min')

# Limpiar y extraer fragmento útil del texto
def extraer_texto_desde_fecha(texto):
    if not isinstance(texto, str):
        return ""
    match = re.search(r"<Paragraph xml:space=\"preserve\">Fecha: \d{2}/\d{2}/\d{4} \d{2}:\d{2}", texto)
    if match:
        return texto[match.start():]
    return texto

df_hc_texto["Texto HC limpio"] = df_hc_texto["Texto HC"].apply(extraer_texto_desde_fecha)

# Frases clave para la lógica secundaria
conducta_keywords = [
    "se indica", "se ordena", "se solicita", "se administra",
    "inicia tratamiento", "se realiza", "se traslada", "se remite",
    "inicio de", "se formula", "orden médica", "se aplica", "se suspende",
    "se avisa a", "se informa a", "se comunica con", "se notifica a"
]

# Funciones auxiliares
def contiene_conducta(texto):
    if isinstance(texto, str):
        texto = texto.lower()
        return any(frase in texto for frase in conducta_keywords)
    return False

def conducta_medica_reportada(texto):
    if not isinstance(texto, str):
        return False
    texto = texto.lower()
    if "se recibe llamada" in texto:
        match = re.search(r"conducta médica tomada:\s*(.+)", texto)
        if match:
            contenido = match.group(1).strip()
            return contenido and not contenido.lower().startswith("no ")
    return False

# Servicios excluidos
servicios_excluidos = {"RADIOLOGIA E IMAGENES DIAGNOSTICAS", "BACTERIOLOGIA"}

# Procesar cada historia clínica
fechas_toma_conducta = []

for _, fila in df_valores_criticos.iterrows():
    historia = fila["Historia clinica"]
    fecha_nota = fila["Fecha nota"]

    notas_historia = df_hc_texto[df_hc_texto["Historia clinica"] == historia]
    if len(notas_historia) <= 1:
        fechas_toma_conducta.append(np.nan)
        continue

    nota_evento = notas_historia[
        (notas_historia["Fecha nota HC"] == fecha_nota) &
        (notas_historia["ESPNOM"] == "RADIOLOGIA E IMAGENES DIAGNOSTICAS")
    ]
    fecha_evento = nota_evento["Fecha nota HC"].iloc[0] if not nota_evento.empty else fecha_nota

    notas_posteriores = notas_historia[notas_historia["Fecha nota HC"] > fecha_evento].sort_values("Fecha nota HC")

    # 1. Lógica prioritaria: conducta médica explícita con encabezado
    conducta_nota = notas_posteriores[
        (notas_posteriores["PRONOM"] == "REPORTE VALORES CRÍTICOS") &
        (~notas_posteriores["ESPNOM"].isin(servicios_excluidos)) &
        (notas_posteriores["Texto HC limpio"].apply(conducta_medica_reportada))
    ]

    if not conducta_nota.empty:
        fecha_conducta = conducta_nota["Fecha nota HC"].iloc[0]
        fechas_toma_conducta.append(fecha_conducta)
        continue

    # 2. Lógica de respaldo: búsqueda por palabras clave
    respaldo_nota = notas_posteriores[
        (~notas_posteriores["ESPNOM"].isin(servicios_excluidos)) &
        (notas_posteriores["Texto HC limpio"].apply(contiene_conducta))
    ]

    if not respaldo_nota.empty:
        fecha_conducta = respaldo_nota["Fecha nota HC"].iloc[0]
        fechas_toma_conducta.append(fecha_conducta)
    else:
        fechas_toma_conducta.append(np.nan)

# Asignar al DataFrame
df_valores_criticos["Fecha toma conducta"] = fechas_toma_conducta

# Exportar resultado
df_valores_criticos.to_excel("Resultado_conducta_Radiología.xlsx", index=False)
print("✅ Archivo generado: Resultado_conducta.xlsx")


print(df_valores_criticos.head(10))